#**This Notebook Shows How to use Agents to Interact with Pandas DataFrame**

#**In LangChain, the Agent can access many dataframes at once**

#**Step 01: Install All the Required Packages**

In [1]:
!pip install langchain -q
!pip install watermark -q
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [5]:
!pip install --quiet langchain_experimental langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


#**Step 02: Import All The Required Libraries**

In [2]:
import os
import warnings
warnings.filterwarnings("ignore")

In [6]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [7]:
# from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.llms import OpenAI

In [8]:
import pandas as pd

#**Step 03: Setup the Environment**


In [10]:
# os.environ["OPENAI_API_KEY"] = "sk-"
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

#**Step 04: Visualize the Data**

In [11]:
url = "https://raw.githubusercontent.com/sudarshan-koirala/Logistic-Regression-for-Titanic-Dataset/master/Train_Titanic.csv"
df=pd.read_csv(url)
print(df.shape)
df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#**Step 05: Single DataFrame Example**

#**Agent can interact with Single DataFrame**

In [15]:
llm=OpenAI(openai_api_key = OPENAI_API_KEY)# temperature=0.7)

In [16]:
agent=create_pandas_dataframe_agent(llm, df, verbose=True)

In [17]:
agent.run("How many rows are there")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I should use the len() function to get the number of rows
Action: python_repl_ast
Action Input: len(df)
Observation: 891
Thought: I now know the final answer
Final Answer: 891 

> Finished chain.


'891'

In [18]:
df[df['SibSp'] > 3].shape

(30, 12)

In [19]:
agent.run("How many people have more than 3 siblings")



> Entering new AgentExecutor chain...
Thought: We need to filter the dataframe for rows where the SibSp column is greater than 3, then count the number of rows.
Action: python_repl_ast
Action Input: df[df['SibSp'] > 3].count()
Observation: PassengerId    30
Survived       30
Pclass         30
Name           30
Sex            30
Age            23
SibSp          30
Parch          30
Ticket         30
Fare           30
Cabin           0
Embarked       30
dtype: int64
Thought: This gives us a series with the count of each column. We only need one value, so we can select any column to get the count.
Action: python_repl_ast
Action Input: df[df['SibSp'] > 3].count()['PassengerId']
Observation: 30
Thought: We now have the count of passengers with more than 3 siblings.
Final Answer: 30

> Finished chain.


'30'

#**Step 06: Multi DataFrame Example**

#**Agent can also interact with Multiple DataFrames passed in a list**

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [21]:
df1=df.copy()

In [22]:
df1["Age"] = df1["Age"].fillna(df1["Age"].mean())

In [24]:
df['Age'].isna().sum()

177

In [25]:
891-177

714

In [ ]:
agent=create_pandas_dataframe_agent(llm, [df, df1], verbose=True)

In [23]:
agent.run("How many rows in the age column are different")



> Entering new AgentExecutor chain...
Thought: I need to compare the values in each row of the age column to determine how many are different.
Action: python_repl_ast
Action Input: df['Age'].nunique()
Observation: 88
Thought: This gives me the number of unique values in the age column, but I need the number of rows that are different.
Action: python_repl_ast
Action Input: len(df['Age'].unique())
Observation: 89
Thought: This gives me the total number of unique values in the age column, which means there is one row with a null value.
Action: python_repl_ast
Action Input: len(df['Age']) - len(df['Age'].unique())
Observation: 802
Thought: This gives me the number of rows in the age column that are different from each other.
Final Answer: There are 802 rows in the age column that are different.

> Finished chain.


'There are 802 rows in the age column that are different.'

In [31]:
agent=create_pandas_dataframe_agent(llm, [df, df1], verbose=True)
agent.run("find the names which contain 'Allen' and their details")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the name contains 'Allen', and then display all the columns for those rows.
Action: python_repl_ast
Action Input: df[df['Name'].str.contains('Allen')]
Observation: NameError: name 'df' is not defined
Thought: I need to specify which dataframe to use, which is df1 and df2.
Action: python_repl_ast
Action Input: df1[df1['Name'].str.contains('Allen')]
Observation:      PassengerId  Survived  Pclass                           Name     Sex  \
4              5         0       3       Allen, Mr. William Henry    male   
730          731         1       1  Allen, Miss. Elisabeth Walton  female   

      Age  SibSp  Parch  Ticket      Fare Cabin Embarked  
4    35.0      0      0  373450    8.0500   NaN        S  
730  29.0      0      0   24160  211.3375    B5        S  
Thought: I only need to display the columns for the name and details, so I can use the .loc function to specify which col

"The names containing 'Allen' and their details in df1 are: Allen, Mr. William Henry and Allen, Miss. Elisabeth Walton."

In [32]:
agent=create_pandas_dataframe_agent(llm, [df, df1], verbose=True)
agent.run("find the names of the passenger who gives max and min Fare")



> Entering new AgentExecutor chain...
Thought: I need to use the .loc[] function to filter the dataframe by the max and min Fare values, and then use the .Name column to get the names of the passengers
Action: python_repl_ast
Action Input: df.loc[df['Fare'] == df['Fare'].max(), 'Name']
Observation: NameError: name 'df' is not defined
Thought: I need to specify the dataframe name
Action: python_repl_ast
Action Input: df1.loc[df1['Fare'] == df1['Fare'].max(), 'Name']
Observation: 258                      Ward, Miss. Anna
679    Cardeza, Mr. Thomas Drake Martinez
737                Lesurer, Mr. Gustave J
Name: Name, dtype: object
Thought: I got the names for the maximum Fare, now I need to do the same for the minimum Fare
Action: python_repl_ast
Action Input: df1.loc[df1['Fare'] == df1['Fare'].min(), 'Name']
Observation: 179                 Leonard, Mr. Lionel
263               Harrison, Mr. William
271        Tornquist, Mr. William Henry
277         Parkes, Mr. Francis "Frank"
302     

'The names of the passenger who gives max Fare are Ward, Miss. Anna; Cardeza, Mr. Thomas Drake Martinez; and Lesurer, Mr. Gustave J. The names of the passenger who gives min Fare are Leonard, Mr. Lionel; Harrison, Mr. William; Tornquist, Mr. William Henry; Parkes, Mr. Francis "Frank"; Johnson, Mr. William Cahoone Jr; Cunningham, Mr. Alfred Fleming; Campbell, Mr. William; Frost, Mr. Anthony Wood "Archie"; Johnson, Mr. Alfred; Parr, Mr. William Henry Marsh; Watson, Mr. Ennis Hastings; Knight, Mr. Robert J; Andrews, Mr. Thomas Jr; Fry, Mr. Richard; and Reuchlin, Jonkheer. John George.'

In [34]:
df1.loc[df1['Fare'] == df1['Fare'].max(), 'Name'].values

array(['Ward, Miss. Anna', 'Cardeza, Mr. Thomas Drake Martinez',
       'Lesurer, Mr. Gustave J'], dtype=object)

In [26]:
df2=df1.copy()

In [27]:
df2["Age_Multiplied"]=df1["Age"] * 2

In [28]:
df2.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Multiplied
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,44.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,76.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,52.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,70.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,70.0


In [29]:
agent=create_pandas_dataframe_agent(llm, [df,df1, df2], verbose=True)

In [30]:
agent.run("Are the number of columns same in all the dataframe")



> Entering new AgentExecutor chain...
Thought: We can check the number of columns by checking the shape of each dataframe.
Action: python_repl_ast
Action Input: df1.shape
Observation: (891, 12)
Thought: We need to check the shape of all dataframes to see if the number of columns are the same.
Action: python_repl_ast
Action Input: df2.shape
Observation: (891, 12)
Thought: The shape is the same for df1 and df2, let's check df3.
Action: python_repl_ast
Action Input: df3.shape
Observation: (891, 13)
Thought: The shape is not the same for df3, so the number of columns are not the same for all dataframes.
Final Answer: No, the number of columns is not the same for all dataframes.

> Finished chain.


'No, the number of columns is not the same for all dataframes.'